# Read and visualize spatial data from a database (MariaDB)

## Install requirements


Create and activate conda environment.
```
conda create -n geoviz python==3.12
conda activate geoviz
```

Install and start mariadb.
```
brew install mariadb
brew services start mariadb
```

Install pip dependencies.
```
pip install mariadb numpy pandas geopandas shapely matplotlib folium mapclassify
```

Set up mariadb access rights.
```
sudo mysql_secure_installation
```


## Start coding

Create connect and disconnect function to make the coding easier later on.

In [65]:
import mariadb
import sys

def connect():
    # Connect to MariaDB Platform
    try:
        conn = mariadb.connect(
            user="geoviz",
            password="geoviz",
            host="localhost",
            port=3306,
            database="geoviz"

        )
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        sys.exit(1)

    # Get Cursor
    cur = conn.cursor()
    return cur, conn

def disconnect(cur, conn):
    # Close connection
    cur.close()
    conn.close()
    

Define functions for creating the database and table.

Also, define utility functions for querying the db.

In [113]:
def create_db():
  cur, conn = connect()
  cur.execute("CREATE DATABASE IF NOT EXISTS geoviz")
  disconnect(cur, conn)

def create_table():
  cur, conn = connect()
  # Create table with spatial column
  cur.execute("""
    CREATE TABLE IF NOT EXISTS geoviz (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255),
        geom GEOMETRY NOT NULL,
        SPATIAL INDEX(geom)
    )
    """)
  disconnect(cur, conn)

def clear_table():
  cur, conn = connect()
  # Create table with spatial column
  cur.execute("TRUNCATE TABLE geoviz;")
  disconnect(cur, conn)

def get_all_rows():
    cur, conn = connect()
    cur.execute("SELECT id, name, ST_AsText(geom) AS geom FROM geoviz")
    rows = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    disconnect(cur, conn)
    return rows, columns

In [117]:
from shapely.geometry import Point


def insert_data():
    cur, conn = connect()
    coords = [
        (13.186, 55.707), # Lund, Sweden
    ]
    for pair in coords:
        point = Point(pair)
        wkt = point.wkt

        # Insert point into the table
        insert_query = "INSERT INTO geoviz (name, geom) VALUES (%s, ST_GeomFromText(%s))"
        cur.execute(insert_query, ("Example Point", wkt))
        conn.commit()
    disconnect(cur, conn)

clear_table()
insert_data()

# Verify insertion
rows, cols = get_all_rows()
print('Columns:', cols)
print('Rows:', [row for row in rows])


Columns: ['id', 'name', 'geom']
Rows: [(1, 'Example Point', 'POINT(13.186 55.707)')]


### Read points and plot a map

In [116]:
import geopandas as gpd
import pandas as pd
# import mysql.connector
from shapely import wkt

def get_df():
  # Fetch data and convert to GeoDataFrame
  rows, cols = get_all_rows()
  df = pd.DataFrame(rows, columns=cols)
  print(df)
  df['geom'] = df['geom'].apply(wkt.loads)
  gdf = gpd.GeoDataFrame(df, geometry='geom')
  gdf = gdf.set_crs(epsg=4326)
  return gdf

gdf = get_df()
gdf.explore(color='red', figsize=(10, 10), cmap="Set1", style_kwds=dict(color="yellow", weight=4, radius=12))

   id           name                  geom
0   1  Example Point  POINT(13.186 55.707)
